In [1]:
import math

def lat_lon_to_tile(lat, lon, zoom):
    """
    Convert a latitude and longitude to tile coordinates for a given zoom level.
    
    Args:
        lat (float): Latitude in degrees.
        lon (float): Longitude in degrees.
        zoom (int): Zoom level, corresponding to the number of divisions along each axis.
                    For example, for a 8x8 grid, use zoom=3 (since 2^3 = 8).
    
    Returns:
        (int, int): Tuple of (x_tile, y_tile) representing the tile coordinates.
    """
    n = 2 ** zoom  # Number of tiles per axis

    x_tile = int((lon + 180.0) / 360.0 * n)
    y_tile = int((1.0 - math.asinh(math.tan(math.radians(lat))) / math.pi) / 2.0 * n)

    return x_tile, y_tile

# Example usage
lat = 40.7128  # Latitude of New York City
lon = -74.0060  # Longitude of New York City
zoom = 3  # Corresponding to an 8x8 grid

x_tile, y_tile = lat_lon_to_tile(lat, lon, zoom)

print(f"The tile for latitude {lat} and longitude {lon} at zoom level {zoom} is: ({x_tile}, {y_tile})")


The tile for latitude 40.7128 and longitude -74.006 at zoom level 3 is: (2, 3)


In [2]:
def global_coords_to_tile(lat, lon, grid_size=8):
    """
    Calculate the tile indices for a given latitude and longitude on a global grid.

    Args:
        lat (float): Latitude in degrees.
        lon (float): Longitude in degrees.
        grid_size (int): The number of tiles per row and per column (8 for an 8x8 grid).

    Returns:
        (int, int): A tuple containing the row and column indices of the tile.
    """
    # Ensure the latitude and longitude are within their bounds
    lat = max(min(lat, 90), -90)
    lon = max(min(lon, 180), -180)

    # Calculate the relative position of the lat and lon on the grid
    lat_pos = (lat + 90) / 180  # Normalize to a 0-1 range
    lon_pos = (lon + 180) / 360  # Normalize to a 0-1 range

    # Calculate the tile indices
    row_index = int(lat_pos * grid_size)
    col_index = int(lon_pos * grid_size)

    # Adjust indices if they are on the edge of the grid
    if row_index == grid_size:
        row_index -= 1
    if col_index == grid_size:
        col_index -= 1

    return (row_index, col_index)

# Example usage for New York City
lat_nyc = 40.7128  # Latitude
lon_nyc = -74.0060  # Longitude
tile_nyc = global_coords_to_tile(lat_nyc, lon_nyc, 8)

print(f"New York City is in tile: {tile_nyc}")


New York City is in tile: (5, 2)


In [3]:
def lat_lon_to_tile_index(lat, lon, grid_rows=8, grid_cols=8):
    """
    Convert latitude and longitude to a grid tile index, starting from the top-left corner.

    Args:
        lat (float): Latitude in degrees (-90 to 90).
        lon (float): Longitude in degrees (-180 to 180).
        grid_rows (int): Number of rows in the grid.
        grid_cols (int): Number of columns in the grid.

    Returns:
        (int, int): A tuple (row_index, col_index) representing the tile's position.
    """
    # Latitude index calculation
    # Convert lat to a 0 (top) to 1 (bottom) scale relative to the grid
    lat_relative = (90 - lat) / 180  # 0 at the top, 1 at the bottom

    # Longitude index calculation
    # Convert lon to a 0 (left) to 1 (right) scale relative to the grid
    lon_relative = (lon + 180) / 360  # 0 on the left, 1 on the right

    # Calculate the tile indices
    row_index = int(lat_relative * grid_rows)
    col_index = int(lon_relative * grid_cols)

    # Ensure indices are within the grid bounds
    row_index = min(max(row_index, 0), grid_rows - 1)
    col_index = min(max(col_index, 0), grid_cols - 1)

    return row_index, col_index

# Example: New York City coordinates
lat = 40.7128  # Latitude
lon = -74.0060  # Longitude

row_index, col_index = lat_lon_to_tile_index(lat, lon, 8, 8)

# Formatting to match your naming convention
tile_name = f"{row_index:02d}_{col_index:02d}_zoom3_urban_fraction_100m.tif"
print(f"The tile for latitude {lat} and longitude {lon} is: {tile_name}")


The tile for latitude 40.7128 and longitude -74.006 is: 02_02_zoom3_urban_fraction_100m.tif


In [5]:
lon, lat = -78.79,-4.30
row_index, col_index = lat_lon_to_tile_index(lat, lon, 8, 8)

# Formatting to match your naming convention
tile_name = f"02_{row_index:02d}_{col_index:02d}_zoom3_urban_fraction_100m.tif"
print(f"The tile for latitude {lat} and longitude {lon} is: {tile_name}")


The tile for latitude -4.3 and longitude -78.79 is: 02_04_02_zoom3_urban_fraction_100m.tif


In [6]:
# Define four pairs of latitude and longitude
locations = [
    (40.7128, -74.0060),  # New York City
    (34.0522, -118.2437),  # Los Angeles
    (51.5074, -0.1278),  # London
    (-33.8688, 151.2093)  # Sydney
]

# Loop through each location, calculate its tile, and print the result
for lat, lon in locations:
    row_index, col_index = lat_lon_to_tile_index(lat, lon, 16, 16)
    tile_name = f"{row_index:02d}_{col_index:02d}_zoom4_urban_fraction_100m.tif"
    print(f"The tile for latitude {lat} and longitude {lon} at zoom level 4 is: {tile_name}")


The tile for latitude 40.7128 and longitude -74.006 at zoom level 4 is: 04_04_zoom4_urban_fraction_100m.tif
The tile for latitude 34.0522 and longitude -118.2437 at zoom level 4 is: 04_02_zoom4_urban_fraction_100m.tif
The tile for latitude 51.5074 and longitude -0.1278 at zoom level 4 is: 03_07_zoom4_urban_fraction_100m.tif
The tile for latitude -33.8688 and longitude 151.2093 at zoom level 4 is: 11_14_zoom4_urban_fraction_100m.tif


In [4]:
def lat_lon_to_tile_index_adjusted(lat, lon, grid_rows=16, grid_cols=16):
    """
    Convert latitude and longitude to a grid tile index, considering the dataset's specific latitude range.
    
    Args:
        lat (float): Latitude in degrees.
        lon (float): Longitude in degrees.
        grid_rows (int): Number of rows in the grid.
        grid_cols (int): Number of columns in the grid.

    Returns:
        (int, int): A tuple (row_index, col_index) representing the tile's position.
    """
    # Adjust the latitude range from +84 to -60
    lat_min = -60
    lat_max = 84

    # Normalize latitude and longitude to a 0-1 scale based on the dataset's coverage
    lat_relative = (lat - lat_max) / (lat_min - lat_max)  # Adjusted for specific lat range
    lon_relative = (lon + 180) / 360  # 0 on the left (west), 1 on the right (east)

    # Calculate the tile indices
    row_index = int(lat_relative * grid_rows)
    col_index = int(lon_relative * grid_cols)

    # Ensure indices are within the bounds of the grid
    row_index = min(max(row_index, 0), grid_rows - 1)
    col_index = min(max(col_index, 0), grid_cols - 1)

    return row_index, col_index

# Example usage
locations = [
    (40.7128, -74.0060),  # New York City
    (34.0522, -118.2437),  # Los Angeles
    (51.5074, -0.1278),    # London
    (-33.8688, 151.2093),   # Sydney
    (45.18377, 5.725210)   # Grenoble
]

for lat, lon in locations:
    row_index, col_index = lat_lon_to_tile_index_adjusted(lat, lon, 64, 64)
    tile_name = f"{row_index:02d}_{col_index:02d}_zoom4_urban_fraction_100m.tif"
    print(f"The tile for latitude {lat} and longitude {lon} at zoom level 4 is: {tile_name}")


The tile for latitude 40.7128 and longitude -74.006 at zoom level 4 is: 19_18_zoom4_urban_fraction_100m.tif
The tile for latitude 34.0522 and longitude -118.2437 at zoom level 4 is: 22_10_zoom4_urban_fraction_100m.tif
The tile for latitude 51.5074 and longitude -0.1278 at zoom level 4 is: 14_31_zoom4_urban_fraction_100m.tif
The tile for latitude -33.8688 and longitude 151.2093 at zoom level 4 is: 52_58_zoom4_urban_fraction_100m.tif
The tile for latitude 45.18377 and longitude 5.72521 at zoom level 4 is: 17_33_zoom4_urban_fraction_100m.tif


In [1]:
from osgeo import gdal

# List of TIFF files to merge
tiff_files = [
    '00_00_zoom4_urban_fraction_100m.tif',
    '00_01_zoom4_urban_fraction_100m.tif',
    '01_00_zoom4_urban_fraction_100m.tif',
    '01_01_zoom4_urban_fraction_100m.tif'
]

# Output file name
output_file = 'python_merged_zoom4_urban_fraction_100m.tif'

# Create a GDAL virtual file system that holds the list of files to be merged
vrt_options = gdal.BuildVRTOptions(resampleAlg='nearest', addAlpha=False, VRTNodata=0)
vrt_filename = gdal.BuildVRT('/vsimem/temp.vrt', tiff_files, options=vrt_options)

# Open the VRT file
vrt_ds = gdal.Open(vrt_filename)

# Finally, use gdal_translate to convert the VRT to the actual merged TIFF
gdal.Translate(output_file, vrt_ds, options="-co COMPRESS=LZW")

# Clean up
vrt_ds = None
gdal.Unlink(vrt_filename)

print(f'Merged file created: {output_file}')



ModuleNotFoundError: No module named 'osgeo'

In [5]:
import requests
import zipfile
import io

# URL of the ZIP file containing the TIFF
zip_file_url = "https://github.com/jacobogabeiraspenas/UrbanSurfAce/raw/main/data/urban_fraction/zoom_4/02_14_zoom4_urban_fraction_100m_int8.zip"

def download_and_extract_zip(zip_url):
    # Send a GET request to the URL
    response = requests.get(zip_url)
    # Check if the request was successful
    if response.status_code == 200:
        # Open the ZIP file contained in the response's bytes
        with zipfile.ZipFile(io.BytesIO(response.content)) as thezip:
            # Extract all the contents into the current directory
            thezip.extractall()
        print("File successfully downloaded and extracted.")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")

# Use the function
download_and_extract_zip(zip_file_url)


File successfully downloaded and extracted.


In [8]:
ls -lh

total 5272896
-rw-r--r--     1 gabeiras3j  staff   2.6M Mar 19 16:26 02_14_zoom4_urban_fraction_100m_int8.tif
-rw-r--r--     1 gabeiras3j  staff    96B Mar 18 18:53 1
-rwxr-xr-x     1 gabeiras3j  staff   872B Mar 16 09:22 2split_tif.sh*
drwxr-xr-x    18 gabeiras3j  staff   576B Mar 16 10:13 FU_tiles_zoom2/
drwxr-xr-x    68 gabeiras3j  staff   2.1K Mar 16 10:12 FU_tiles_zoom3/
-rw-r--r--     1 gabeiras3j  staff    26B Feb 28 14:58 Makefile
drwxr-xr-x     5 gabeiras3j  staff   160B Feb 28 15:01 Notebooks/
drwxr-xr-x  3333 gabeiras3j  staff   104K Mar 13 14:30 OSM_tiles/
-rw-r--r--     1 gabeiras3j  staff   2.1K Feb 28 14:58 README.md
-rw-r--r--     1 gabeiras3j  staff    14K Mar 18 18:21 Tiles_FU.ipynb
-rw-r--r--     1 gabeiras3j  staff   3.0K Mar 12 17:42 Untitled.ipynb
-rw-r--r--     1 gabeiras3j  staff    57K Mar 12 13:06 Untitled1.ipynb
-rw-r--r--     1 gabeiras3j  staff   381K Mar 13 11:51 Untitled2.ipynb
-rw-r--r--     1 gabeiras3j  staff   126K Mar 14 10:40 Untitled3.ipynb
-rw-r--